```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
import pandas as pd
import os

base_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 2


excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3)[['관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
# import shutil
# from tqdm import tqdm

# zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
# original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

# error_list = []

# for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
#    fn = str(fn)
#    if '/' in lv: 
#       lv = lv.replace("/", '_')
   
#    if 'Lv' not in lv:
#       continue

#    # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
#    if lv == 'Lv2':
#       # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
#       continue
#    else:
#       os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

#       file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
#       try:
#          with open(file_path, 'r', encoding='utf-8') as f:
#                content = f.read()
         
#          # Replace the old filename with new filename
#          updated_content = content.replace(str(fn), str(tn))
         
#          with open(file_path, 'w', encoding='utf-8') as f:
#                f.write(updated_content)
               
#          print(f"Updated {fn} to {tn} in {file_path}")
#       except Exception as e:
#          print(f"Error processing {fn} to {tn}: {e}")
#          error_list.append((fn, tn, lv))
#          # 이름 바꾸기
#          # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

In [89]:
import shutil
from tqdm import tqdm
import os, json

original_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/2C/Lv5"
final_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C/Lv5"

for file in os.listdir(original_path):
    if not file.startswith('SS'):
        continue

    with open(os.path.join(final_path, file, file+'.json.bak'), 'r', encoding='utf-8') as f:
        data = json.load(f)

    with open(os.path.join(original_path, file, file+'.json'), 'r', encoding='utf-8') as f:
        new_data = json.load(f)

    # str_data = str(data)
    # str_new_data = str(new_data)
    # 'contents' 위치 찾기
    # contents_pos_new = str_new_data.find('contents')
    # contents_pos_data = str_data.find('contents')

    # 슬라이싱을 사용하여 'contents' 이전 부분만 교체
    # result = str_data[:contents_pos_data] + str_new_data[contents_pos_new:]
    # break
    with open(os.path.join(final_path, file, file+'.json'), 'w', encoding='utf-8') as f:
        json.dump(new_data, f, ensure_ascii=False, indent=4)


    # os.system(f"cd {original_path} && mv -f {file}/{file}.json Lv5/{file}/ && rm -rf Lv5/{file}/crop && mv -f {file}/crop Lv5/{file}/")
    # os.system(f"cd {original_path}/{file} && rmdir pdf_page && cd .. && rmdir {file}")

### 업로드

In [ ]:
# import os

# # from_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/mock_exam/test" 
# from_path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/tools/QueryModels.py'
# to_path = "yjmoon@oci004:/home/yjmoon/src"


# # 업로드
# os.system(f"scp -r {from_path} {to_path}")
# # for ss in file:
#     # os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
# for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
#     fn = str(fn)
#     if '/' in lv: 
#        lv = lv.replace("/", '_')
# #     print(fn, tn, lv)
    
#     dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
#     os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
#     print(os.path.join(dir, fn))
#     # break

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

## 전체 돌리기

In [90]:
import tools.ProcessFiles as pf

excel = pf.get_excel_data(2)
excel

,ISBN,도서명,출판일,코퍼스 1분류,코퍼스 2분류,비고,분류
관리번호,,,,,,,
SS0001,9791196259976,"보험, 진짜 필요할까? : 보험을 제대로 모르는 사람들을 위한 또박또박 보험가이드",2018-03-29,보험,기초,시사,Lv2
SS0002,9791197848629,5분에 만나는 표준약관 : 생명보험,2022-04-15,보험,실무,이론,Lv3/4
SS0003,9791197848612,5분에 만나는 표준약관 : 질병·상해보험(손해보험 회사용),2022-04-15,보험,실무,이론,Lv3/4
SS0004,9791198375902,주식 가치평가를 위한 작은 책,2023-07-20,증권,기초,시사,Lv3/4
SS0005,9791192312620,세계 인플레이션 시대의 경제지표,2023-10-19,금융,기초,시사,Lv3/4
...,...,...,...,...,...,...,...
SS0264,9788960543201,알면 돈되는 보험 상품 100% 활용법,2023-09-20,보험,기초,보험계약,Lv3/4
SS0265,9791198974433,2025 감정평가사 객관식 민법,2024-11-04,보험,수험서,감정평가사,Lv3/4
SS0266,9791198804082,2025 감정평가사 부동산학원론 객관식,2024-09-03,보험,수험서,감정평가사,Lv3/4


In [93]:
import json
import tools.ProcessLv2 as pl2

path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv5'
for file in os.listdir(path):
    if not file.startswith('SS'):
        continue

    INPUT_PATH = os.path.join(path, file, file+'.json')
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    # with open(BACKUP_PATH, "w", encoding="utf-8") as f:
    #     json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # break
    id = os.path.splitext(os.path.basename(file))[0]
    try:
        new = {
            'file_id': id,
            'ISBN': str(excel.loc[id, 'ISBN']),
            'title': excel.loc[id, '도서명'],
            'cat1_domain': excel.loc[id, '코퍼스 1분류'],
            'cat2_sub': excel.loc[id, '코퍼스 2분류'],
            'cat3_specific': excel.loc[id, '비고'],
            'pub_date': str(excel.loc[id, '출판일'])[:10],
            # 'contents': [],
            'contents': origin['contents']
        }
    except:
        new = origin
    
    # break
    # new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    # new = pl2.erase_page(origin, 1)
    # new = pl2.format_change(3,origin)
    # new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
    # # with open(INPUT_PATH.replace(".json", "_new.json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)

## Lv2 pdf_page 만들기

In [ ]:
# import pandas as pd
# import os

# base_path = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
# analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
# buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
# i = 3

# excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3, engine='openpyxl')[['관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
# lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
# lv
# # print(lv == 'Lv2')

In [ ]:
# # Fixed version - using Python file operations instead of sed
# import shutil
# from tqdm import tqdm
# import fitz

# dpi = 300
# mat = fitz.Matrix(dpi / 150, dpi / 150)

# # for tn, isbn, lv in tqdm(zip(excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
# for tn in tqdm(excel_analy['관리번호']):
#    # if '/' in lv: 
#    #    lv = lv.replace("/", '_')
   
#    # if 'Lv' not in lv:
#    #    continue
#    lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
#    # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
#    if lv == 'Lv2':
#       os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2 && mkdir {tn} && mv {tn}*.json {tn}/ && cd {tn} && mkdir pdf_page")
#       # pdf_page 만들기
#       pdf_path = f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv1/{tn}.pdf"
#       doc = fitz.open(pdf_path)
#       for p, page in enumerate(doc):
#          img = page.get_pixmap(matrix=mat)
#          img.save(f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2/{tn}/pdf_page/{tn}_{p+1:04d}.png")
      
#       # break

#    # SS0041 - SS0045 사이에서 MuPDF error: syntax error: invalid key in dict

### 개별처리

# Fin-Know 평가

In [ ]:
import pandas as pd
import re

def remove_think_block(text):   ## for Qwen3
    if isinstance(text, str):
        return re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL)
    return text

model_name = 'claude-3.7-sonnet'

res_file = f"/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/fai_code/eval_fai/result/res_{model_name}_fai-fin-know-2504a.csv"

df_res = pd.read_csv(res_file)
df_res = df_res.fillna(" ")
df_res['response_rev'] = df_res['response'].apply(remove_think_block)
df_res['response_rev'] = df_res['response_rev'].map(lambda x: "".join(re.findall(r"[a-zA-Z]", x)))
df_res['response_rev'] = df_res['response_rev'].map(lambda x: x.lower().strip()[-1] if x != "" else x)

df_res['correct'] = (df_res['answer'] == df_res['response_rev'])

# df_to_write = df_res[["answer", "response_rev", "response", "correct"]]
# df_to_write.to_csv("thinking_answer.csv")
precision = round(df_res['correct'].sum()/len(df_res)*100, 2)
print(precision)

In [ ]:
df_res[df_res['response'] == " "]

In [ ]:
df_res[df_res['correct'] == False][['answer', 'response']]

In [ ]:
import pandas as pd
import os, sys
import json

model_name = 'res_claude-3.7-sonnet_fai-fin-know-2504a'
path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/fai_code/eval_fai/result'

# JSON에서 파일 목록 로드
merge_files = [file for file in os.listdir(path) if file.startswith(model_name)]
print(merge_files[0])
print(f"[병합] 총 {len(merge_files)}개 파일을 병합합니다...")

dfs = []
for i, file_path in enumerate(merge_files):
    try:
        df = pd.read_csv(os.path.join(path, file_path), encoding='utf-8-sig')
        print(f"[병합] 그룹 {i}: {len(df)}개 행 로드 완료 - {file_path}")
        dfs.append(df)
    except Exception as e:
        print(f"[병합] 오류: {file_path} 로드 실패 - {e}")
        sys.exit(1)

if len(dfs) == 0:
    print("[병합] 오류: 병합할 데이터가 없습니다!")
    sys.exit(1)

# 모든 데이터프레임 병합
merged_df = pd.concat(dfs, ignore_index=True)
print(f"[병합] 총 {len(merged_df)}개 행으로 병합 완료")

# 병합된 파일 저장
merged_df.to_csv(os.path.join(path, model_name+".csv"), index=False, encoding='utf-8-sig')
print(f"[병합] 병합된 파일 저장: {model_name}")
print(f"[병합] ✓ 병합 완료!")